<a href="https://colab.research.google.com/github/saintblue97/DeepNN_Optiver/blob/Brett_validation_work/brett_10_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive; drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
#!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v optiver-trading-at-the-close        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
# !kaggle competitions leaderboard --show                       # print public leaderboard

- competition is now set to: optiver-trading-at-the-close
100% 201M/201M [00:03<00:00, 52.8MB/s]


In [3]:
%%time
%%capture
%reset -f
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, xgboost, lightgbm as lgb, librosa

ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=300): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 2.39 s, sys: 350 ms, total: 2.74 s
Wall time: 4.84 s


In [4]:
def is_google_colab():
    """Check if the environment is Google Colab.

    Returns:
        bool: True if in Google Colab, False otherwise.
    """
    try:
        import google.colab
        return True
    except ImportError:
      return False

if is_google_colab():
  file_path = ''
else:
  file_path = '/kaggle/input/optiver-trading-at-the-close/'

In [5]:
sample_submission_path = file_path + 'example_test_files/sample_submission.csv'

sample_submission = pd.read_csv(sample_submission_path); sample_submission

,time_id,row_id,target
0,26290,478_0_0,1
1,26290,478_0_1,1
...,...,...,...
32998,26454,480_540_198,1
32999,26454,480_540_199,1


# Load Data

In [6]:
tmr = Timer()

⏳ started. You have 300 sec. Good luck!


In [7]:
train_data_path = file_path + 'train.csv'
df = pd.read_csv(train_data_path);
df = df.dropna(subset=['target']); df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.18e+06,1,1.0,1.34e+07,NaN,NaN,1.0,60651.50,1.0,8493.03,1.0,-3.03,0,0_0_0
1,1,0,0,1.67e+05,-1,1.0,1.64e+06,NaN,NaN,1.0,3233.04,1.0,20605.09,1.0,-5.52,0,0_0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237978,198,480,540,1.00e+06,1,1.0,9.48e+07,1.0,1.0,1.0,125631.72,1.0,669893.00,1.0,-1.54,26454,480_540_198
5237979,199,480,540,1.88e+06,-1,1.0,2.41e+07,1.0,1.0,1.0,250081.44,1.0,300167.56,1.0,-6.53,26454,480_540_199


In [8]:
test_data_path = file_path + 'example_test_files/test.csv'
df_test  = pd.read_csv(test_data_path); df_test

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id
0,0,478,0,3.75e+06,-1,1.0,1.15e+07,NaN,NaN,1.0,22940.00,1.0,9177.60,1.0,26290,478_0_0
1,1,478,0,9.86e+05,-1,1.0,3.85e+06,NaN,NaN,1.0,1967.90,1.0,19692.00,1.0,26290,478_0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32998,198,480,540,1.00e+06,1,1.0,9.48e+07,1.0,1.0,1.0,125631.72,1.0,669893.00,1.0,26454,480_540_198
32999,199,480,540,1.88e+06,-1,1.0,2.41e+07,1.0,1.0,1.0,250081.44,1.0,300167.56,1.0,26454,480_540_199


In [9]:
revealed_targets_path = file_path + 'example_test_files/revealed_targets.csv'
df_revealed_targets  = pd.read_csv(revealed_targets_path); df_revealed_targets



,stock_id,date_id,seconds_in_bucket,time_id,revealed_target,revealed_date_id,revealed_time_id
0,0.0,478,0,26290,-2.31,477.0,26235.0
1,1.0,478,0,26290,-12.85,477.0,26235.0
...,...,...,...,...,...,...,...
33160,NaN,480,530,26453,NaN,NaN,NaN
33161,NaN,480,540,26454,NaN,NaN,NaN


In [10]:
print('Shape of training data =', df.shape)
print('Shape of testing data  =', df_test.shape)

Shape of training data = (5237892, 17)
Shape of testing data  = (33000, 16)


In [11]:
tX, tY = df.drop('target', axis=1), df.target    # full training set

X_train = tX
y_train = tY

# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(tX, tY, test_size=0.2)
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3)

# Preprocessing

In [12]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler,OneHotEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import ARDRegression, BayesianRidge, LinearRegression, Ridge, Lars

In [13]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Only drop columns that exist in X
        columns_to_drop = [col for col in self.columns_to_drop if col in X.columns]
        return X.drop(columns_to_drop, axis=1)

class DataFrameSimpleImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean', fill_value=None):
        self.strategy_ = strategy
        self.fill_value_ = fill_value

    def fit(self, X, y=None):
        if self.strategy_ == 'mean':
            self.fill_values_ = X.mean()
        elif self.strategy_ == 'median':
            self.fill_values_ = X.median()
        elif self.strategy_ == 'constant':
            if self.fill_value_ is None:
                raise ValueError("fill_value must be provided for strategy='constant'")
            self.fill_values_ = pd.Series(self.fill_value_, index=X.columns)
        else:
            raise ValueError(f"Unknown strategy type: {self.strategy_}")

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill_values_)

class MovingAverageTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, window_size, columns):
        self.window_size = window_size  # The size of the window for moving average
        self.columns = columns          # The columns on which to apply moving average

    def fit(self, X, y=None):
        return self  # No fitting process, so just return self

    def calculate_moving_average(self, series):
        return series.rolling(window=self.window_size).mean()

    def transform(self, X):
        X_new = X.copy()

        for col in self.columns:
            X_new[f'{col}_sma_{self.window_size}'] = X_new.groupby('stock_id')[col].transform(self.calculate_moving_average)

            X_new[f'{col}_sma_{self.window_size}'].fillna(X_new[col], inplace=True)


        return X_new


In [14]:
from sklearn.compose import ColumnTransformer

# Define ColumnTransformer for one-hot encoding
one_hot_encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['stock_id', 'imbalance_buy_sell_flag']),
    ],
)

# Set up preprocessing pipeline
preprocess_steps = [
  ('drop_columns', DropColumns(['near_price','far_price','time_id'])),
  # ('imputer', SimpleImputer(strategy='median')),
  ('imputer', DataFrameSimpleImputer(strategy='constant', fill_value=0)),
  ('moving_average', MovingAverageTransformer(window_size=2, columns=['ask_size','ask_price','bid_price','ask_size','reference_price','wap'])),
  # ('one_hot_encoder', one_hot_encoder),
  # ('poly', PolynomialFeatures(degree=2, include_bias=False)),
  # ('scaler', StandardScaler()),
]
pipeline = Pipeline(preprocess_steps)

# Make submission

In [15]:
lr = LinearRegression()
lr.fit(pipeline.fit_transform(X_train),y_train)

LinearRegression()

In [16]:
precomputed_predictions = lr.predict(pipeline.fit_transform(df_test))

 # Convert lists to dataframes
df_pred = pd.DataFrame(precomputed_predictions, columns=['predicted_target'])

# Make sure both lists are of the same length before computing MAE
min_len = min(len(df_revealed_targets), len(precomputed_predictions))
df_true = df_revealed_targets.head(min_len)
df_pred = df_pred.head(min_len)

# Calculate and print the MAE
mae = mean_absolute_error(df_true['revealed_target'].fillna(0), df_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 5.490227437747271


In [17]:
df_true[:11000]

,stock_id,date_id,seconds_in_bucket,time_id,revealed_target,revealed_date_id,revealed_time_id
0,0.0,478,0,26290,-2.31,477.0,26235.0
1,1.0,478,0,26290,-12.85,477.0,26235.0
...,...,...,...,...,...,...,...
10998,198.0,478,540,26290,1.30,477.0,26289.0
10999,199.0,478,540,26290,1.24,477.0,26289.0


In [18]:
# if not is_google_colab():
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [19]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

# if not is_google_colab():

# Initialize index for precomputed_predictions
pred_idx = 0

# Flag to indicate if it's the first batch
first_batch = True

for (test, revealed_targets, sample_prediction) in iter_test:

    # Get current batch size
    current_batch_size = len(test)
    predictions_batch = lr.predict(pipeline.fit_transform(test))

    # Take the slice from precomputed predictions for the current batch
    precomputed_predictions_batch = precomputed_predictions[pred_idx: pred_idx + current_batch_size]

    predictions_batch = precomputed_predictions_batch.copy()

    # Update the index for the next iteration
    pred_idx += current_batch_size

    # Make the actual prediction and add to sample_prediction df
    sample_prediction['target'] = predictions_batch

    # Submit the predictions for this chunk of test data
    env.predict(sample_prediction)


This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [30]:
predictions_batch

array([-0.01,  0.99,  0.81, -0.09, -0.  , -0.05,  0.99,  0.25,  0.7 ,  0.06, -0.17,  0.62,  0.14,
       -0.27, -0.73,  0.13,  0.35, -0.08,  0.33, -0.27, -0.41, -0.23,  0.02,  0.14,  0.02, -0.03,
        0.69, -0.17, -0.19, -0.59,  0.06, -5.25, -0.27,  0.65,  0.4 ,  0.06, -0.13, -0.04, -0.1 ,
        0.4 ,  0.47, -0.01,  0.22, -0.02,  0.26, -0.11,  0.2 , -0.26, -0.02, -0.03,  0.17, -0.12,
       -0.29,  0.29,  0.12, -0.11, -0.13,  0.39,  0.29,  0.05,  0.63,  1.61,  0.68,  0.13,  0.13,
       -0.15, -0.03, -0.3 , -0.17, -0.66, -0.79,  0.29,  0.02,  0.55,  0.65,  0.05, -0.05, -1.09,
       -1.22,  0.08,  0.8 ,  0.27,  3.51, -0.14, -0.11,  1.9 ,  0.53,  0.15,  0.03,  0.39,  0.78,
       -0.05,  2.09,  0.88, -0.13,  0.13,  0.13,  0.12, -0.19,  0.07,  1.59,  1.55, -0.08, -0.18,
        0.06, -0.17, -0.13,  0.06,  1.63, -0.17, -0.07, -2.05, -0.17, -0.38, -0.13,  0.31, -0.13,
       -0.74, -0.14, -2.93,  0.78,  0.13,  0.29, -0.03, -0.3 ,  0.09,  0.15,  1.01, -0.1 , -0.09,
        0.41, -0.38,

In [21]:
df_revealed_targets

,stock_id,date_id,seconds_in_bucket,time_id,revealed_target,revealed_date_id,revealed_time_id
0,0.0,478,0,26290,-2.31,477.0,26235.0
1,1.0,478,0,26290,-12.85,477.0,26235.0
...,...,...,...,...,...,...,...
33160,NaN,480,530,26453,NaN,NaN,NaN
33161,NaN,480,540,26454,NaN,NaN,NaN


In [22]:
if not is_google_colab():

    # Initialize index for precomputed_predictions
    pred_idx = 0

    # Flag to indicate if it's the first batch
    first_batch = True

    for (test, revealed_targets, sample_prediction) in iter_test:

        # Get current batch size
        current_batch_size = len(test)

        # Take the slice from precomputed predictions for the current batch
        current_predictions = precomputed_predictions[pred_idx: pred_idx + current_batch_size]

        # Update the index for the next iteration
        pred_idx += current_batch_size

        # Make the actual prediction and add to sample_prediction df
        sample_prediction['target'] = current_predictions

        # Submit the predictions for this chunk of test data
        env.predict(sample_prediction)

In [23]:
tmr.ShowTime()    # measure Colab's runtime.

Runtime is 139 sec
